In [1]:
import sys; sys.path.append('../src/')
from pointnetkl import *

In [2]:
# load our best network
net = torch.load('../models/1000x4_all.net', map_location='cpu')

In [3]:
# our datasets
datasets = ['../data/consolidated/{}_val.npy'.format(name) for name in ['antarctica', 'borno']]

In [4]:
# for each dataset
for name in datasets:
    
    # load the dataset
    d = Data([name])

    # put the dataset into a PyTorch loader
    d = torch.utils.data.DataLoader(d, batch_size=10)
    
    # list of losses for each sample
    losses = list()

    # loop through batches because memory is a problem :'(
    for submap, prior, posterior, rho, submap_fn, covariance_fn in d:

        # get the posterior predictions
        posterior_p = net(submap, prior, rho)

        # compute the losses per sample for this batch
        loss = Model.kl_loss(posterior, posterior_p)

        # add the losses to the main loss list
        [losses.append(l.item()) for l in loss]

    # average the losses
    losses = sum(losses)/len(losses)
    
    # print message
    print('The loss for {} is {}'.format(name, losses))

Loading ../data/consolidated/antarctica_val.npy
The loss for ../data/consolidated/antarctica_val.npy is 3.4958412691459553
Loading ../data/consolidated/borno_val.npy
The loss for ../data/consolidated/borno_val.npy is 0.37351390169326054
